In [ ]:
# Clone code from repository
%cd /content/drive/MyDrive
!git clone https://github.com/duydinh92/WSSS.git

In [ ]:
# Change path to clone folder
%cd /content/drive/MyDrive/WSSS

In [ ]:
# Install requirements
!pip install -r requirements.txt

In [ ]:
# Unzip dataset
!unzip /content/drive/MyDrive/WSSS/VOC2012_train_val.zip -d /content/

In [ ]:
# Train ResNet with classification task
!python classification/train_res+er+pcm.py --voc12_root /content/VOC2012_train_val --train_list voc12/train_aug.txt --val_list voc12/train.txt

In [ ]:
# Infer class activation map from trained ResNet (if want to use trained model, put train_res152+er+pcm_crop512_lr0.01_epoch8.pth in given link to folder experiments/models)
!python classification/infer_cam.py --voc12_root /content/VOC2012_train_val --infer_list voc12/train_aug.txt --weights experiments/models/train_res152+er+pcm_crop512_lr0.01_epoch8.pth

In [ ]:
# Train AffinityNet with label generate from previous cams with high confidence object and high confidence background(if you want to use pretrained backbone model, put )
!python classification/train_affinity.py --voc12_root /content/VOC2012_train_val --train_list voc12/train_aug.txt --ho_crf_dir crf_4.0 --hb_crf_dir crf_24.0 

In [ ]:
# Improve previous cams with trained AffinityNet to generate pseudo label for segmentation (if want to use trained model, put train_affinity_net8.pth in given link to folder experiments/models)
!python classification/infer_affinity.py --voc12_root /content/VOC2012_train_val --weights experiments/models/train_affinity_net8.pth --infer_list voc12/train_aug.txt --out_label pseudo_label

In [ ]:
# Evaluate generated pseudo label for segmentation
!python classification/evaluation_cam.py --list voc12/train.txt --voc12_root /content/VOC2012_train_val --predict_dir pseudo_label

In [ ]:
# Train DeepLabV3+ backbone ResNet101 with pseudo label 
!python segmentation/main.py --data_root /content/VOC2012_train_val --model deeplabv3plus_resnet101

In [ ]:
# Evaluate trained DeepLabV3+ model in val data (if want to use trained model, put best_deeplabv3plus_resnet101_pseudo.pth in given link to folder experiments/models)
!python segmentation/main_eval.py --data_root /content/VOC2012_train_val --dataset voc --test_only --model deeplabv3plus_resnet101 --ckpt experiments/models/best_deeplabv3plus_resnet101_pseudo.pth

In [ ]:
# Use trained DeepLabV3+ model to predict mask
!python segmentation/predict.py --model deeplabv3plus_resnet101 --input input_dir --dataset voc --ckpt experiments/models/best_deeplabv3plus_resnet101_pseudo.pth --save_val_results_to result_dir

In [ ]:
# Install package to run streamlit in colab
!npm install localtunnel

In [ ]:
# Run streamlit app 
!streamlit run app.py &>/dev/null&
!npx localtunnel --port 8501